# Getting Started with Cryptography in Python

#### Crypto & Privacy Village (DEFCON 24), 2016

[Amirali Sanatinia](http://www.ccs.neu.edu/home/amirali)

 - Cryptography is ubiquitous today
 - From mobile phones to wireless communication
 - Supported in almost every programming language 
 - It is even embedded in the CPUs
 - It is not hard to do crypto right but...
 
 <img src="include/crypto_failures.png">


We will look at the basic blocks of modern crypto using Python. There are a number of crypto libraries in Python:

 - **pycrypto** (oldest and most widely used)
 - **m2crypto** (SWIG binding)
 - **cryptography** (PY2, PY3, PyPy, OpenSSL CFFI binding)


We will use the cryptography library. You can download the library from [here](https://cryptography.io/) and follow the instructions. You should be able to install the library using the following command. You need to have pip insalled. 

```bash
pip install cryptography
```

To install pip, follow the instructions [here](https://pip.pypa.io/)

*note: Each code block has extra imports, so that blocks would be independent runnable code*

## Cyrptography.io
Cryptography components are divided into different submodules. Following is a list of these submodules (not exhaustive)

* Primitive Crypto Blocks (*cryptography.hazmat*)
 * Message Digest and Hashing algorithms (*cryptography.hazmat.primitives.hashes*)
 * Symmetric encryption algorithms (*cryptography.hazmat.primitives.ciphers*)
 * Asymmetric encryption algorithms (*cryptography.hazmat.primitives.asymmetric*)
* X.509 Ecosystem (*cryptography.x509*)
* Full high level crypto recipe (*cryptography.fernet*)

## Hashing Algorithms

 - **Input**: long message
 - **Output**: short block (called hash or message digest)
 - **Desired properties**: 
  - Pre-image: Given a hash *h* it is computationally infeasible to find a message *m* that produces *h*
  - Second preimage: Given message m, it is computationally infeasible to find a message m’, (m ≠ m’) such that, h(m) = h(m’)
  - Collisions: It is computationally difficult to find any two messages m, m’ (m ≠ m’) such that, h(m) = h(m’)

**Examples**:
 - Recommended Hash Algorithm (SHA-2, SHA-3) by NIST
 - SHA-1: output 160 bits being phased out 
 - MD2, MD4, and MD5 by Ron Rivest [RFC1319, 1320, 1321]


### SHA Family

Secure Hash Algorithm (SHA) family, is a series of hashing algorithms. Ranging from SHA-0 to SHA-3. SHA-0 should never be used, it's advised to move from SHA-1 to SHA-2. SHA-3 is the most recent version, published in 2015.

 * SHA-1: Digest size (160), Block size (512)
 * SHA-2: Digest size (224, 256, 384, or 512), Block size (512, 1024)
 * SHA-3: Digest size (224, 256, 384, 512), Block size (1600)

In [ ]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes
import base64
for _hash in [hashes.SHA1, hashes.SHA224, hashes.SHA256, hashes.SHA384, hashes.SHA512]:
    digest = hashes.Hash(_hash(), backend=default_backend())
    digest.update(b"CryptoVillage")
    digest.update(b"2016")
    msg_digest = digest.finalize()
    # Notice the output size of the digest
    print _hash.name, len(msg_digest), len(msg_digest) * 8, base64.b64encode(msg_digest)

### Hash-based message authentication code (HMAC)

HMACs are used for message authentications combined with a secret key. The provide integrity check and authentication.

<img src="include/SHAhmac.png">

image source: wikipedia

In [ ]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hmac, hashes
import os
import base64

hmc_key = k = os.urandom(16)
hmc = hmac.HMAC(hmc_key, hashes.SHA1(), default_backend())
hmc.update("CryptoVillage2016")
hmc_sig = hmc.finalize()
print base64.b64encode(hmc_sig)

In [ ]:
hmc = hmac.HMAC(hmc_key, hashes.SHA1(), default_backend())
hmc.update("CryptoVillage2016")
hmc.verify(hmc_sig)

In [ ]:
hmc = hmac.HMAC(hmc_key, hashes.SHA1(), default_backend())
hmc.update("CryptoVillage2016")
hmc.verify("Wrong_Signature")

<img src="include/sym_vs_asym.png">

### RSA

RSA, is an asymmetric encryption algorithm by Ron Rivest, Adi Shamir, and Leonard Adleman. It was published in 1977. It's security is based on the hardness of factorization problem. However, now it has its own problem, called the RSA problem. RSA is slow, and is not used for encryptin large data, but it's mostly used to encrypt the symmetric key that is used for encryption.


 * p, q, two big prime numbers (private, chosen)
 * n = pq, φ(n) = (p-1)(q-1)   (public, calculated)
 * e, with gcd(φ(n), e) = 1,  1 < e < φ(n)	(public, chosen)
 * d = e - 1 mod φ(n)	(private, calculated)
 * $E(M) = M^e \mod n$
 * $D(M) = M^d \mod n$
 * $D(E(M)) = M^{ed} \mod n = M$

## RSA EXAMPLE

 - p = 5; q = 11 => n = 55
 - φ(n) = 40
 - e = 3 => d = 27
  - Because ed = 1 mod φ(n)
 - Public key: (e, n)
 - Private key: (d, n)
 - Encryption
  - M = 2
 - Encryption(M) = $ M^e\mod n$  = $2^3\mod n$ = 8
 - Decryption(8) = $ M^d\mod n$  = $8^{27} \mod n$ = 2

In [ ]:
2 ** 3 % 55

In [ ]:
8 ** 27 % 55

### OpenSSL

To generate keys, use the following instructions:

```bash
 openssl genrsa -out private_key.pem 2048
 openssl pkcs8 -topk8 -inform PEM -outform DER -in private_key.pem -out private_key.der -nocrypt
 openssl rsa -in private_key.pem -pubout -outform DER -out public_key.der
 ```
 

In [ ]:
%%bash
openssl genrsa -out private_key.pem 2048
openssl pkcs8 -topk8 -inform PEM -outform DER -in private_key.pem -out private_key.der -nocrypt
openssl rsa -in private_key.pem -pubout -outform DER -out public_key.der

In [ ]:
# import key from a file. E.g., previously generated by OpenSSL
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import serialization

with open("private_key.pem", "rb") as key_file:
     private_key = serialization.load_pem_private_key(
            key_file.read(),
            password=None,
            backend=default_backend())
public_key = private_key.public_key()

In [ ]:
# Generate a 2048 bit private key
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.asymmetric import rsa

private_key = rsa.generate_private_key(
    public_exponent=65537,
    key_size=2048,
    backend=default_backend())
# to get the public key
public_key = private_key.public_key()

In [ ]:
print bin(2**16 + 1)
print bin(2**1 + 1)

## It's all about padding

Textbook RSA is not IND-CPA secure, therefore we use Optimal Asymmetric Encryption Padding (OAEP). There are also other attacks against RSA with improper padding

<img src="include/RSA_OAEP.png">

image souce: wikipedia

In [ ]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import padding

message = b"The SECRET KEY"
ciphertext = public_key.encrypt(
    message,
    padding.OAEP(
        mgf=padding.MGF1(algorithm=hashes.SHA1()),
        algorithm=hashes.SHA1(),
        label=None))

In [ ]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import padding

plaintext = private_key.decrypt(
    ciphertext,
    padding.OAEP(
        mgf=padding.MGF1(algorithm=hashes.SHA1()),
        algorithm=hashes.SHA1(),
        label=None))

## Symmetric Encryption

In the following we look at the symmetric encryption algorithms. In symmetric crpto, we use the same key for encryption and decryption. Therefore, the two parties needs to establish a secret key between them. It's up to 1000 times faster than asymmetric encryption.


### Advanced Encryption Algorithm (AES)

AES is based on Rijndael encryption algorithm, designed by Joan Daemen and Vincent Rijmen. It was one of the algorithms submitted to U.S. National Institute of Standards and Technology (NIST) to replace DES and 3DES. It was published in 1998 and accepted and standardized in 2001.

 * AES supports key sizes of 128/192/256 bits
 * Block size: 128 bit
 * It's iterative rather than Feistel cipher
 * Treats data in 4 groups of 4 bytes
 * Operates on an entire block in every round
 * Resistant against known attacks
 * Speed and code compactness on many CPUs
 * Rijndael block and key size vary between 128, 192, 256
 * However, in AES block size in 128
 * Number of rounds a function of key size
  * 128 bits     10 rounds
  * 192 bits     12 rounds
  * 256 bits     14 rounds

 * Today most implementations use the CPU support (Intel AES-NI)

### Block cipher mode of operation

To encrypt messages of arbitrary size with block ciphers, we use the following algorithms, called the modes of operation. They define how to encrypt each block of the plaintext to produce the corresponding cipher text block. Some of these are complemetly insecure (ECB) and should not be used.

 * Electronic Codebook (ECB)
 * Cipher Block Chaining (CBC)
 * Counter (CTR)
 
 
### Electronic Codebook (ECB)

<img src="include/ECB_enc.png">
<img src="include/ECB_dec.png">



### Cipher Block Chaining (CBC)

<img src="include/CBC_enc.png">
<img src="include/CBC_dec.png">



### Counter (CTR)

<img src="include/CTR_enc.png">
<img src="include/CTR_dec.png">

image source: wikipedia

The following images are encrypted with ECB. Note that you can see the pattern in the data. Therefore, ECB is not secure or recommended to be used.

<img src="include/tux.png">
<img src="include/ECB1.png">
<img src="include/ECB2.png">

In [ ]:
import os
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
key = os.urandom(16) # in bytes, 128 bits
iv = os.urandom(16)

In [ ]:
# ECB Mode, we only need a key
### *** DO NOT ECB. IT IS INSECURE *** ###

cipher = Cipher(algorithms.AES(key), modes.ECB(), backend=default_backend())
encryptor = cipher.encryptor()
# note that we don't need padding here, since len("Network Security") = 16
cipher_text = encryptor.update("Network Security") + encryptor.finalize()

In [ ]:
cipher_text

In [ ]:
decryptor = cipher.decryptor()
decryptor.update(cipher_text) + decryptor.finalize()

In [ ]:
# CBC Mode, we also need an IV
cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())
encryptor = cipher.encryptor()
# note that we don't need padding here, since len("Network Security") = 16
cipher_text = encryptor.update("Network Security") + encryptor.finalize()

In [ ]:
cipher_text

In [ ]:
decryptor = cipher.decryptor()
decryptor.update(cipher_text) + decryptor.finalize()

In [ ]:
# CTR Mode, we don't need padding in CTR mode. In transforms a block cipher into a stream cipher
# we only need to introduce the nonce
cipher = Cipher(algorithms.AES(key), modes.CTR(os.urandom(16)), backend=default_backend())
encryptor = cipher.encryptor()
# len("Network Security 2016") = 21, but no padding is needed
cipher_text = encryptor.update("Network Security 2016") + encryptor.finalize()

In [ ]:
cipher_text

In [ ]:
decryptor = cipher.decryptor()
decryptor.update(cipher_text) + decryptor.finalize()

## Bit flipping attack

In [ ]:
iv = os.urandom(16)
cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())
encryptor = cipher.encryptor()
cipher_text = encryptor.update("To:25--From:9367") + encryptor.finalize()

In [ ]:
cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())
decryptor = cipher.decryptor()
decryptor.update(cipher_text) + decryptor.finalize()

In [ ]:
def xor(s1, s2):
    return "".join([chr(ord(a) ^ ord(b)) for a,b in zip(s1,s2)]) 

In [ ]:
iv2 = iv[:3] + xor(xor(iv[3:5], "25"), "80") + iv[5:]

In [ ]:
cipher = Cipher(algorithms.AES(key), modes.CBC(iv2), backend=default_backend())
decryptor = cipher.decryptor()
decryptor.update(cipher_text) + decryptor.finalize()

## Pretty Good Privacy (PGP)

 - A data encryption/decryption tool
 - Can be used to encrypt and authenticat email, files, etc.
 - Created by Phil Zimmermann in 1991
 - A practical hybrid system that uses symmetric and asymmetric crypto

<img src="include/PGP.png">

image source: wikipedia

In [ ]:
def enc_key(public_key, message):
    pass

def dec_key(private_key, ciphertext):
    pass

def enc_msg(key, iv, msg):
    pass

def dec_msg(key, iv, ciphertext):
    pass

def gen_hmac(key, msg):
    pass

def verify_hmac(key, msg, sig):
    pass

In [ ]:
k1 = os.urandom(16)
k2 = os.urandom(16)
iv = os.urandom(16)
msg = "CryptoPrivacy 16"

cipher = enc_msg(k1, iv, msg)
sig = gen_hmac(k2, msg)
encrypted_key = enc_key(public_key, k1)

decrypted_key = dec_key(private_key, encrypted_key)
plaintext = dec_msg(decrypted_key, iv, cipher)
verify_hmac(k2, plaintext, sig)